In [ ]:
from __future__ import print_function, division, absolute_import
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline
import astropy.io.ascii as at
from astropy import table


import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (8,5)
mpl.rcParams['lines.markeredgewidth'] = 1.5
mpl.rcParams['lines.markersize'] = 8
mpl.rcParams['font.size'] = 16
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.minor.size'] = 3
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.minor.size'] = 3
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['ytick.minor.width'] = 1
mpl.rcParams['ytick.labelsize'] = 18
mpl.rcParams['legend.frameon'] = False
mpl.rcParams['legend.numpoints'] = 1
mpl.rcParams['legend.fontsize'] = 16
# mpl.rcParams[''] = 

home_dir = os.getenv("HOME")

In [ ]:
# Define values from Meibom+2005

cp_rows = [("PMS binaries",-2.5,7.1,1.2,1.2),
           ("Pleiades",-1,7.2,1.8,1.9),
           ("M35",-0.8,9.9,1.2,1.2),
           ("HyPra",np.log10(0.600),3.2,1.2,1.2),
           ("M67",0.6,12.1,1.0,1.5),
           ("NGC188",0.8,14.5,1.4,2.2),
           ("NGC 6819",np.log10(2.5),6.2,1.1,1.1), # Milliman2014
           ("Field",0.95,10.3,1.5,3.1),
           ("Halo",1.00,15.6,2.3,3.2)
          ]

cp_table = table.Table(rows=cp_rows, 
                       names=('Name', 'log(age)', 'CP','up_unc','dn_unc'), 
                       meta={'name': 'circularization periods'})

In [ ]:
# new_clusters = ["NGC2099 (M37)", "NGC2516", "NGC3532", "NGC6866", "NGC6811", "Rup147"]
# new_ages = [np.log10(0.550),8.052-9,8.492-9,8.576-9,0,np.log10(3)]
# new_clusters = ["NGC6866", "NGC2099 (M37)", "NGC6811", "Rup147"]
# new_ages = [np.log10(0.5),np.log10(0.550),0,np.log10(2.6)]
# new_clusters = ["NGC2099 (M37)", "Praesepe"]
# new_ages = [np.log10(0.550),np.log10(0.600)]
new_clusters = ["Praesepe", "NGC6811"]
new_ages = [np.log10(0.600),0]


new_table = table.Table([new_clusters,new_ages], names=('Name', 'log(age)'), meta={'name': 'new clusters'})

print(10**new_table["log(age)"]*1000)

In [ ]:
cp_errs = np.vstack([cp_table["dn_unc"],cp_table["up_unc"]])
print(cp_errs)

In [ ]:
# Witte & Savongjie 2002
ws_age = np.array([0.01,0.1,0.8,4,11])
ws_dn = np.array([2,3,4,6,10])
ws_up = np.array([2,5,5,6,10])

In [ ]:
# Geller 2013
geller = at.read(os.path.join(home_dir,"HyPra/models/geller2013_fig2_combined_arohatgi_2016-10-01.csv"))

In [ ]:
def plot_cp(new_table,prop_label="Proposed Targets"):
    plt.figure()
    ax = plt.subplot()

    ax.set_xscale("log")
    ax.set_xlim(1e-3,1.1e1)
    ax.set_ylim(0,20)
    ax.set_xticklabels(["",-3,-2,-1,0,1])
    # ax.tick_params(labelsize="x-large")
    ax.set_xlabel("log(age (Gyr))")
    ax.set_ylabel("Circularization Period")

    # Zahn & Bouchet 1989 as cited in Meibom 2005

    pms_r = Rectangle((1e-3,7.2), 3e1, 1.3, color="lightgrey",
                    zorder=-111,ec="none",label="Zahn & Bouchet (1989; PMS only)")
    pms = ax.add_patch(pms_r)

    # Witte & Savongjie 2002

    witte = ax.fill_between(ws_age,ws_dn,ws_up,color='grey',hatch="/",lw=2,
                            label="Witte & Savonjie (2002; Dyn. Tide)",zorder=-100)

    # Zahn 2008 (ref'ing Zahn 19XX)

    zahn_age = np.logspace(-2,1,20)
    zahn_cp = 6 * (zahn_age/5.0)**(3/16)
    zahn = ax.plot(zahn_age,zahn_cp,'k-.',lw=3,label="Zahn (2008; Eq. Tide)",zorder=2)

    # # Goodman & Dickson 1998
    # # Effectively the same as the Zahn prediction for the equilibrium tide (!?)
    # goodman_cp = 6.4 * (zahn_age/10)**(1/7)
    # goodman = ax.plot(zahn_age,goodman_cp,'b-.',lw=4,label="Goodman & Dickson (1997; Dyn. Tide)",zorder=2)


    # Geller 2013 (combines Hurley+2002 and Kroupa 1995)
    gel = ax.plot(geller["t(Gyr)"],geller["CP"],"k:",lw=3,label="Geller+2013 (Hybrid Model)",zorder=1)

    # Data!

    meibom = ax.errorbar(10**cp_table["log(age)"],cp_table["CP"],cp_errs,lw=0,
                         elinewidth=2.5,ms=12,marker="o",capsize=0,mec="none",color="k",
                         label="Meibom & Matheiu (2005) (updated)",zorder=100)

    for row in new_table:
        ax.axvline(10**row["log(age)"],color="m",lw=3,ls="--")
    # for i in np.array([1,3,6]):
    #     ax.axvline(10**cp_table["log(age)"][i],color="m",lw=1.5,ls="--")

    # Adding separate Praesepe line bc will be splitting HyPra up
    newfgk = ax.axvline(100,color="m",lw=3,ls="--",label=prop_label,zorder=101) 

    # hypraplei = np.array([1,3,6])
    # newm = ax.plot(10**cp_table["log(age)"][hypraplei],cp_table["CP"][hypraplei],"m*",
    #                 ms=18,label="Additional RVs",zorder=102)

    handles, labels = ax.get_legend_handles_labels()
    print(handles,labels)
    reorder = np.array([5,2,3,0,4,1])

    new_handles = [handles[i] for i in reorder]

    ax.legend(handles=new_handles,
        loc=2,numpoints=1,borderaxespad=0.1,fontsize=13.5)

In [ ]:
def proposal_plot(new_clusters,new_ages,new_label,outfilename):
    new_table = table.Table([new_clusters,new_ages], 
                             names=('Name', 'log(age)'), 
                             meta={'name': 'new clusters'})
    plot_cp(new_table,prop_label=new_label)
    plt.savefig(outfilename,bbox_inches="tight")
   

In [ ]:
proposal_plot(["Praesepe", "NGC6811"],[np.log10(0.600),0],"Proposed Clusters","cp_prae_6811.pdf")

In [ ]:
proposal_plot(["Praesepe"],[np.log10(0.600)],"Praesepe (Proposed Targets)","cp_prae.pdf")

In [ ]:
proposal_plot(["Praesepe", "Ruprecht 147"],[np.log10(0.600),np.log10(2.5)],"Proposed Clusters","cp_prae_R147.pdf")